In [144]:
import pandas as pd

In [145]:
pred_data = pd.read_csv("/Users/tyleremanuel/CSV Files/viz_data.csv")

In [146]:
pd.options.display.max_columns = None

In [147]:
all_plays = []
for index, row in pred_data.iterrows():
    combo = (row['gameId'], row['playId'])
    if combo not in all_plays:
        all_plays.append(combo)

In [148]:
all_plays

[(2022090800, 56),
 (2022090800, 80),
 (2022090800, 101),
 (2022090800, 122),
 (2022090800, 146),
 (2022090800, 167),
 (2022090800, 191),
 (2022090800, 212),
 (2022090800, 299),
 (2022090800, 343),
 (2022090800, 393),
 (2022090800, 414),
 (2022090800, 486),
 (2022090800, 529),
 (2022090800, 569),
 (2022090800, 593),
 (2022090800, 617),
 (2022090800, 646),
 (2022090800, 692),
 (2022090800, 775),
 (2022090800, 818),
 (2022090800, 933),
 (2022090800, 1030),
 (2022090800, 1102),
 (2022090800, 1187),
 (2022090800, 1230),
 (2022090800, 1254),
 (2022090800, 1334),
 (2022090800, 1358),
 (2022090800, 1385),
 (2022090800, 1406),
 (2022090800, 1712),
 (2022090800, 1736),
 (2022090800, 1836),
 (2022090800, 1946),
 (2022090800, 1967),
 (2022090800, 2043),
 (2022090800, 2072),
 (2022090800, 2093),
 (2022090800, 2163),
 (2022090800, 2184),
 (2022090800, 2208),
 (2022090800, 2336),
 (2022090800, 2360),
 (2022090800, 2464),
 (2022090800, 2485),
 (2022090800, 2506),
 (2022090800, 2527),
 (2022090800, 25

In [149]:
frame_per_play = []
for index, row in pred_data.iterrows():
    if (row['gameId'], row['playId']) in all_plays and row['tackle'] == 1.0 and (row['points_saved'] >= 0 or 0 > row['points_saved']):
        frame_per_play.append(row)
        all_plays.remove((row['gameId'], row['playId']))
        


In [150]:
final_data = pd.DataFrame(frame_per_play)

In [151]:
len(final_data)

7887

In [152]:
final_data.to_csv("final_tackler.csv", index = False)

In [153]:
player_tackles_points = {}
avg_saved = {}
for index, row in final_data.iterrows():
    if row['displayName'] not in player_tackles_points:
        player_tackles_points[row['displayName']] = {'tackles': 0, 'points_saved': 0, 'team': row['club']}
        avg_saved[row['displayName']] = {'pts_saved_per_tackle': 0, 'team': row['club']}

In [154]:
player_tackles_points

{'Jalen Ramsey': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Ernest Jones': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Troy Hill': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Bobby Wagner': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Leonard Floyd': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Terrell Lewis': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Gregory Rousseau': {'tackles': 0, 'points_saved': 0, 'team': 'BUF'},
 'Tremaine Edmunds': {'tackles': 0, 'points_saved': 0, 'team': 'BUF'},
 'Tim Settle': {'tackles': 0, 'points_saved': 0, 'team': 'BUF'},
 'Dane Jackson': {'tackles': 0, 'points_saved': 0, 'team': 'BUF'},
 'Jordan Poyer': {'tackles': 0, 'points_saved': 0, 'team': 'BUF'},
 'Nick Scott': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Justin Hollins': {'tackles': 0, 'points_saved': 0, 'team': 'LA'},
 'Micah Hyde': {'tackles': 0, 'points_saved': 0, 'team': 'BUF'},
 'Ed Oliver': {'tackles': 0, 'points_saved': 0, 'team': 'BUF'},
 'D

In [155]:
for index, row in final_data.iterrows():
    player_tackles_points[row['displayName']]['tackles'] += 1
    player_tackles_points[row['displayName']]['points_saved'] += row['points_saved']

In [156]:
for player in player_tackles_points:
    avg_saved[player]['tackles'] = player_tackles_points[player]['tackles']
    avg_saved[player]['pts_saved_per_tackle'] = player_tackles_points[player]['points_saved']/ player_tackles_points[player]['tackles']

In [157]:
for player in player_tackles_points:
    if player_tackles_points[player]['team'] == 'NO':
        print(player_tackles_points[player]['points_saved'])

26.94415375782218
6.244140451511214
1.312701835879379
6.276095270761783
12.499722672340855
6.218789431575641
5.104760086552486
12.321493336051132
4.768808500937138
11.520245853522646
3.000558760104294
9.596972281594864
8.400825998257236
6.069334057724952
4.731805711431114
8.572725032465314
1.9273261999577025
2.391565204162067
3.339066213076614
1.657476247427215
4.414819303130981
0.2632957341354675


In [158]:
# takes in a team and returns their most impactful tackler in terms of points saved per tackle
# (minimum 10 tackles)
def player_impact_by_team(team):
    team_tacklers = {}
    best_tackler = ''
    max_save = -1e9
    for player in avg_saved:
        if avg_saved[player]['team'] == team:
            team_tacklers[player] = avg_saved[player]
    for player in team_tacklers:
        if team_tacklers[player]['pts_saved_per_tackle'] > max_save and team_tacklers[player]['tackles'] > 10:
            best_tackler = player
            max_save = team_tacklers[player]['pts_saved_per_tackle']
    return best_tackler, max_save

In [159]:
teams = []
for player in avg_saved:
    if avg_saved[player]['team'] not in teams:
        teams.append(avg_saved[player]['team'])

In [164]:
team_stats = {}
for team in teams:
    team_stats[team] = {'total_tackles': 0, 'pts_saved': 0}

In [165]:
for player in avg_saved:
    team = avg_saved[player]['team']
    team_stats[team]['total_tackles'] += avg_saved[player]['tackles']
    team_stats[team]['pts_saved'] += player_tackles_points[player]['points_saved']
team_stats

{'LA': {'total_tackles': 254, 'pts_saved': 133.12706796535497},
 'BUF': {'total_tackles': 236, 'pts_saved': 138.41665714479007},
 'NO': {'total_tackles': 245, 'pts_saved': 147.57668194042228},
 'ATL': {'total_tackles': 278, 'pts_saved': 120.40397546784088},
 'CLE': {'total_tackles': 211, 'pts_saved': 113.1106535713791},
 'CAR': {'total_tackles': 276, 'pts_saved': 147.39935777554956},
 'SF': {'total_tackles': 201, 'pts_saved': 112.96214611956172},
 'CHI': {'total_tackles': 266, 'pts_saved': 119.80989218014409},
 'CIN': {'total_tackles': 254, 'pts_saved': 140.7918648672986},
 'PIT': {'total_tackles': 259, 'pts_saved': 139.79656553011188},
 'PHI': {'total_tackles': 226, 'pts_saved': 126.73713831579322},
 'DET': {'total_tackles': 263, 'pts_saved': 113.28470441315258},
 'HOU': {'total_tackles': 259, 'pts_saved': 135.27986624634187},
 'IND': {'total_tackles': 289, 'pts_saved': 173.05445750531044},
 'MIA': {'total_tackles': 253, 'pts_saved': 150.53465671460293},
 'NE': {'total_tackles': 222, 

In [166]:
team_tackle_impact = {}
for team in team_stats:
    team_tackle_impact[team] = team_stats[team]['pts_saved'] / team_stats[team]['total_tackles']

In [171]:
# this variable represents each team and their average points saved per tackle
team_tackle_impact

{'LA': 0.5241223148242321, 'BUF': 0.5865112590880935, 'NO': 0.6023538038384583, 'ATL': 0.43310782542388804, 'CLE': 0.5360694482055882, 'CAR': 0.5340556441143101, 'SF': 0.5620007269629936, 'CHI': 0.4504131284967823, 'CIN': 0.5542986805799158, 'PIT': 0.5397550792668412, 'PHI': 0.5607837978574921, 'DET': 0.4307403209625573, 'HOU': 0.5223160858932119, 'IND': 0.5988043512294479, 'MIA': 0.594998643140723, 'NE': 0.6177522046235117, 'BAL': 0.439986386918141, 'NYJ': 0.5556152227841791, 'TEN': 0.3944295209328922, 'NYG': 0.43405651562692776, 'WAS': 0.8044364131911708, 'JAX': 0.5919959612215685, 'ARI': 0.4878554129644703, 'KC': 0.504971005945075, 'LV': 0.5446471411594197, 'LAC': 0.5739796602965648, 'GB': 0.5600506103995279, 'MIN': 0.5188229922916628, 'TB': 0.5402702807676263, 'DAL': 0.6703656976434909, 'DEN': 0.570926200630118, 'SEA': 0.5392818476348212}

('Aaron Donald', 0.879867276387427)
('Taron Johnson', 0.8937303410747216)
('Shy Tuttle', 0.7997476901329054)
("Ta'Quon Graham", 0.73295710417929

In [173]:
# lists each team's most impactful tackler, measured by points saved per tackle
# (minimum 10 tackles)
for team in teams:
    print(player_impact_by_team(team))

('Aaron Donald', 0.879867276387427)
('Taron Johnson', 0.8937303410747216)
('Shy Tuttle', 0.7997476901329054)
("Ta'Quon Graham", 0.7329571041792918)
('Jeremiah Owusu-Koramoah', 0.848446214023651)
('Brian Burns', 1.0252709279679169)
('Samson Ebukam', 1.0871710776997376)
('Armon Watts', 1.0178889942097196)
('Chidobe Awuzie', 0.9040204596322441)
('Arthur Maulet', 1.1102049425761416)
('Josh Sweat', 0.9543971090131349)
('Mike Hughes', 0.5815296726385699)
('Steven Nelson', 0.8224012254301969)
('E.J. Speed', 1.3603429551612016)
('Christian Wilkins', 0.815151242629887)
('Jabrill Peppers', 0.9882645031508445)
('Broderick Washington', 0.6904608815818456)
('Quinnen Williams', 0.9263758787168968)
('Dylan Cole', 0.8229313599813151)
('Jihad Ward', 1.110917377737746)
('Jamin Davis', 1.177181839756935)
('Foyesade Oluokun', 0.8270674273587233)
('Zach Allen', 1.2819898275201436)
('Willie Gay', 1.0489863613155441)
('Maxx Crosby', 1.1673690538864099)
('Khalil Mack', 0.8011567127215148)
('Rashan Gary', 1.06